In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#Rain評級
import urllib
import gzip
import json
import requests
import mysql.connector
import datetime
import threading, time
import numpy as np #python陣列的套件，命名為np
from datetime import datetime
from threading import Thread
from urllib.request import urlretrieve
import re #字串多重分割套件

def func():
    global tStart #開始爬蟲時間
    global tEnd   #結束爬蟲時間
    global timenow#當前時間
    
    print ("crawling web page")
    urlretrieve = "http://opendata.epa.gov.tw/webapi/Data/RainTenMin/?$select=SiteName,County,Township,Rainfall10min,Rainfall1hr,Rainfall3hr,Rainfall6hr,Rainfall12hr,Rainfall24hr,Now,PublishTime&$orderby=PublishTime%20DESC&$skip=0&$top=1000&format=json"
    #氣象局開放資料，篩選需要的欄位後呈現在網頁上
    data = requests.get(urlretrieve).json() #解析網頁內容後將json格式轉換成list
    
    timenow = time.localtime(time.time())
    print(timenow)
    a=np.array(time.localtime(time.time()))#將時間格式轉換成陣列
    date = datetime.now().strftime('%Y_%m_%d')#取出當前時間 年_月_日
    
    if a[1]==10:#當時間來到10月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="rain_10", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==11:#當時間來到11月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="rain_11", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    elif a[1]==12:#當時間來到12月份
        conn = mysql.connector.connect(host="localhost", user="root", passwd="", db="rain_12", charset = 'utf8')#更換存取的db
        c = conn.cursor()
    else:
        print("db not found")
        pass
    
    if True:
        print("1")        
        for i in range(0,len(data)): #跑list長度的迴圈次數，list中每個物件為一個雨量站中所有參數，一次迴圈處理一個雨量站
            data[i] = str(data[i])
            data[i] = data[i].split("'") #字串分割
            d={} #創建字典
            #把值寫入字典
            d['SiteName']=data[i][3]
            d['County']=data[i][7]
            d['Township']=data[i][11]
            d['PublishTime']=data[i][43]
            date=data[i][43]
            date=re.split("'|\s",date)[0] #使用re套件進行多重分割
            date=date.replace('-','_') #把 - 替換成 _
            #print(date)
            d['Rainfall10min']=data[i][15]
            d['Rainfall1hr']=data[i][19]
            d['Rainfall3hr']=data[i][23]
            d['Rainfall6hr']=data[i][27]
            d['Rainfall12hr']=data[i][31]
            d['Rainfall24hr']=data[i][35]
            d['Now']=data[i][39]
            #雨量評級寫入變數
            Rainfall1hr_level=float(data[i][19])
            Rainfall3hr_level=float(data[i][23])
            Rainfall24hr_level=float(data[i][35])
            '''
            #政府雨量評級標準
            Rainfall1hr>=40 Rainfall1hr="大雨"
            Rainfall24hr>=80 Rainfall24hr="大雨"

            Rainfall3hr>=100 Rainfall3hr="豪雨"
            Rainfall24hr>=200 Rainfall24hr="豪雨"

            Rainfall24hr>=350 Rainfall24hr="大豪雨"
            Rainfall24hr>=500 Rainfall24hr="超大豪雨"

            Rainfall24hr=0 Rainfall24hr="無下雨"
            Rainfall24hr<40 Rainfall1hr="小雨"
            '''
            #一小時雨量評級寫入變數
            if Rainfall1hr_level==0:
                Rainlevel1hr = '無下雨'
            elif Rainfall1hr_level>0 and Rainfall1hr_level<40:
                Rainlevel1hr = '小雨'
            elif Rainfall1hr_level>=40:
                Rainlevel1hr = '大雨'
            else:
                Rainlevel1hr = '無資料'
            
            #雨量評級寫入變數
            if Rainfall24hr_level==0:
                Rainlevel24hr = '無下雨'
            elif Rainfall24hr_level>0 and Rainfall24hr_level<40:
                Rainlevel24hr = '小雨'
            elif (Rainlevel1hr == '小雨') and (Rainfall24hr_level>=40 and Rainfall24hr_level<80):
                Rainlevel24hr = '小雨'
            elif (Rainlevel1hr == '大雨') and (Rainfall24hr_level>=40 and Rainfall24hr_level<80):
                Rainlevel24hr = '大雨'
            elif Rainfall24hr_level>=80 and Rainfall24hr_level<200:
                Rainlevel24hr = '大雨'
            elif Rainfall3hr_level>=100 or (Rainfall24hr_level>=200 and Rainfall24hr_level<350):
                Rainlevel24hr = '豪雨'
            elif Rainfall24hr_level>=350 and Rainfall24hr_level<500:
                Rainlevel24hr = '大豪雨'
            elif Rainfall24hr_level>=500:
                Rainlevel24hr = '超大豪雨'
            else:
                Rainlevel24hr = '無資料'

                
            d['Rainlevel1hr']=Rainlevel1hr
            d['Rainlevel24hr']=Rainlevel24hr
            #print(d)
            #將字典的值放入變數
            SiteName = d['SiteName']
            County = d['County']
            Township = d['Township']
            PublishTime = d['PublishTime']
            Rainfall10min = d['Rainfall10min']
            Rainfall1hr = d['Rainfall1hr']
            Rainfall3hr = d['Rainfall3hr']
            Rainfall6hr = d['Rainfall6hr']
            Rainfall12hr = d['Rainfall12hr']
            Rainfall24hr = d['Rainfall24hr']
            Now = d['Now']
            Rainlevel1hr = d['Rainlevel1hr']
            Rainlevel24hr = d['Rainlevel24hr']
            
            sql="CREATE TABLE IF NOT EXISTS " + date + "(`SiteName` VARCHAR(4) CHARACTER SET utf8 NOT NULL , `County` VARCHAR(4) CHARACTER SET utf8 NOT NULL ,`Township` VARCHAR(4) CHARACTER SET utf8 NOT NULL , `PublishTime` DATETIME NOT NULL ,`Rainfall10min` FLOAT(4) NOT NULL ,`Rainfall1hr` FLOAT(4) NOT NULL ,`Rainfall3hr` FLOAT(4) NOT NULL ,`Rainfall6hr` FLOAT(4) NOT NULL ,`Rainfall12hr` FLOAT(4) NOT NULL ,`Rainfall24hr` FLOAT(4) NOT NULL ,`Now` FLOAT(4) NOT NULL ,`Rainlevel1hr` VARCHAR(4) CHARACTER SET utf8 NOT NULL,`Rainlevel24hr` VARCHAR(4) CHARACTER SET utf8 NOT NULL)ENGINE = InnoDB DEFAULT CHARSET=latin1 AUTO_INCREMENT=0;"
            #print(sql) #date以日期名稱命名table
            try:
                c.execute(sql) #創建table
            except Exception as e:
                print("Mysql Error %d: %s" % (e.args[0], e.args[1]))
                pass
            if County == '臺北市': #從全臺雨量站挑出臺北市的，其餘不寫入
                if SiteName == '竹子湖' and Township == '士林區': #如果竹子湖出現在士林區，命名為竹子湖2
                    SiteName='竹子湖2'
                else:
                    pass
                #資料寫入table
                sql = "INSERT INTO " + date + " (SiteName,County,Township,PublishTime,Rainfall10min,Rainfall1hr,Rainfall3hr,Rainfall6hr,Rainfall12hr,Rainfall24hr,Now,Rainlevel1hr,Rainlevel24hr) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            else:
                pass
            try:
                c.execute(sql,(SiteName,County,Township,PublishTime,Rainfall10min,Rainfall1hr,Rainfall3hr,Rainfall6hr,Rainfall12hr,Rainfall24hr,Now,Rainlevel1hr,Rainlevel24hr))
            except:
                pass
            #處理完一個雨量站跑下一個，直到跑完全部的
        
        conn.commit()
        conn.close()

def start():
    t = Thread(target=func) #跑func這個function
    t.start()
   
if __name__ == '__main__':
    counter=0
    a=5
    tStart1 = time.time()
    start() #跑start這個function
    tEnd1 = time.time()
    time.sleep(200-(tEnd1 - tStart1)) #設定每200秒起來檢查一次
    tEnd1 = time.time()
    tstart0=tStart1
    
    while(a>0):
        tStart1 = tEnd1
        
        if((tStart1-tstart0)>=600): #如果到600秒起來爬資料(原始資料更新頻率)
            counter=counter+1
        else:
            tstart0=tEnd1
            counter=counter+1
            
        if(counter%3==0):
            start()
        else:
            print("")    
            
        tEnd1 = time.time()
        time.sleep(200-(tEnd1 - tStart1))        
        tEnd1 = time.time()